In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from time import sleep
from bs4 import BeautifulSoup

In [ ]:
def get_soup_from_url(url):
    # Helper to get response soup from url
    sleep(1)
    try:
        html = urlopen(url).read()
    except HTTPError:
        print("Link Cannot be Reached")
        return -1
    #soup = BeautifulSoup(html,"lxml")
    soup = BeautifulSoup(html, "html.parser")
    return soup


In [ ]:
def get_all_matchids_for_season(season='2018/19'):
    #season = "2018"
    #season = "2018/19"
    season = season.replace('/', '%2F')
    url = "http://www.espncricinfo.com/ci/engine/series/index.html?season={};view=season".format(season)
    soup = get_soup_from_url(url)
    d={}
    for section in soup.find_all('div', class_='match-section-head'):
        series_in_section = [series['data-series-id']
                             for series in section.find_next(
                                 'section',
                                  class_='series-summary-wrap'
                             ).find_all(
                                 'section',
                                  class_="series-summary-block collapsed"
                             )
                            ]
        series_in_section = [
            get_soup_from_url('http://www.espncricinfo.com/ci/engine/match/index/series.html?series={}'.format(s)) 
            for s in series_in_section
        ]
        matches_in_section = [
            re.findall('\/([0-9]+)\/', 
                      m.find('a')['href'])[1] 
            for ss in series_in_section 
            for m in ss.find_all(class_='match-no')
        ]
        d[section.text] = matches_in_section
    return d

In [ ]:
match_ids_2024 = get_all_matchids_for_season('2024')
match_ids = match_ids_2024['Tests']

If You Have matchid.txt From CRICSHEET Which Is A List Of Matches

In [ ]:
# import re

# with open('matchid.txt', 'r') as file:
#     text = file.read()

# match_ids = re.findall(r'\b\d{7}\b', text)

In [26]:
import requests

def get_redirected_commentary_url(match_id, return_type=2):
    # Construct the original match URL
    original_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.html"
    
    # Perform a HEAD request to get the redirected URL
    response = requests.head(original_url, allow_redirects=True)
    redirected_url = response.url

    # Modify the redirected URL for commentary
    if "full-scorecard" in redirected_url:
        commentary_url = redirected_url.replace("full-scorecard", "ball-by-ball-commentary")
    elif "live-cricket-score" in redirected_url:
        commentary_url = redirected_url.replace("live-cricket-score", "ball-by-ball-commentary")
    else:
        raise ValueError("Redirected URL does not contain expected parts ('full-scorecard' or 'live-cricket-score'). Cannot modify to commentary URL.")

    # Return the appropriate URL based on return_type
    if return_type == 1:
        return redirected_url
    elif return_type == 2:
        return commentary_url
    else:
        raise ValueError("Invalid return_type. Use 1 for redirected URL or 2 for commentary URL.")


In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import os
import json
import time
import re

# Setup Chrome options

def setup_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_argument("--disable-plugins")
    options.add_argument("--disable-extensions")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    options.page_load_strategy = 'eager'
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    # Limit network resources
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    # Initialize WebDriver
    service = Service('/usr/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=options)

    return driver

# Dictionary to store different types of API URLs

# 

def capture_and_categorize_urls(driver, api_responses, api_url_types, type):
    logs = driver.get_log('performance')
    for entry in logs:
        try:
            log = json.loads(entry['message'])['message']
            if log.get('method') == 'Network.responseReceived':
                response_url = log.get('params', {}).get('response', {}).get('url', '')
                
                check_str = ''
                if(type == 1):
                    check_str = 'hs-consumer-api.espncricinfo.com/v1/pages/match/scorecard'
                else:
                    check_str = 'hs-consumer-api.espncricinfo.com/v1/pages/match/comments'

                # Check if the URL contains the desired comment API URL
                if check_str in response_url:
                    # Check if the sortDirection is ASC
                    if type == 1 :
                        print(f"Captured Fetch Request URL: {response_url}")
                        
                        # Extract the response body using CDP
                        request_id = log['params']['requestId']
                        
                        response_body = driver.execute_cdp_cmd(
                            'Network.getResponseBody',
                            {'requestId': request_id}
                        )

                        # Add to responses list
                        api_responses.append({
                            "url": response_url,
                            "body": json.loads(response_body.get('body', '{}'))  # Parse JSON body
                        })
                        
                        # Categorize based on the unique identifier in the URL
                        match = re.search(r'/comments/(\w+)', response_url)
                        if match:
                            if match.group(1) == 'default':
                                api_url_types['comments_urls'].add(response_url)
                            else:
                                api_url_types['other_comments_urls'].add(response_url)

                        print(f"Captured Scorecard URL: {response_url}")   

                    elif 'sortDirection=ASC' in response_url:
                        # Process only the URLs where sortDirection=ASC
                        print(f"Captured Fetch Request URL (ASC): {response_url}")
                        
                        # Extract the response body using CDP
                        request_id = log['params']['requestId']
                        response_body = driver.execute_cdp_cmd(
                            'Network.getResponseBody',
                            {'requestId': request_id}
                        )

                        # Add to responses list
                        api_responses.append({
                            "url": response_url,
                            "body": json.loads(response_body.get('body', '{}'))  # Parse JSON body
                        })
                        
                        # Categorize based on the unique identifier in the URL
                        match = re.search(r'/comments/(\w+)', response_url)
                        if match:
                            if match.group(1) == 'default':
                                api_url_types['comments_urls'].add(response_url)
                            else:
                                api_url_types['other_comments_urls'].add(response_url)

                        print(f"Captured Comments URL (ASC): {response_url}")
                    else:
                        print(f"Skipping URL with sortDirection=DESC: {response_url}")
        
        except Exception as e:
            print(f"Error capturing URL: {e}")
            continue

def scroll_page_func(driver, api_responses, api_url_types):
    scroll_increment = 200
    scroll_pause_time = 1
    page_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0

    while current_position < page_height:
        # Scroll down
        driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
        current_position += scroll_increment
        time.sleep(scroll_pause_time)

        # Update page height
        page_height = driver.execute_script("return document.body.scrollHeight")

        # Capture URLs
        capture_and_categorize_urls(driver, api_responses, api_url_types, 2)

    print("Scrolling and URL capture completed.")

def espn_commentary_scrapper_engine(url, match_id):

    api_url_types = {
        'comments_urls': set(),
        'other_comments_urls': set()
    }

    api_responses = []
    

    driver = setup_driver()

    try:
        # Open the target webpage
        driver.get(url)

        time.sleep(2)

        # innings_dropdown = driver.find_element(By.XPATH, "//div[contains(@class, 'ds-popper-wrapper') and .//span[contains(@class, 'ds-text-tight-s')]]")
        innings_dropdown = driver.find_element(By.XPATH, "//div[@class='ds-popper-wrapper']//div[contains(@class, 'ds-flex ds-items-center ds-border-ui-stroke ds-h-6 ds-px-4 ds-border ds-bg-ui-fill ds-rounded-full ds-w-full ds-min-w-max ds-cursor-pointer')]")
        driver.execute_script("arguments[0].click();", innings_dropdown)
        # driver.execute_script("arguments[0].setAttribute('aria-expanded', 'true');", innings_dropdown)
        
        # time.sleep(2)
          # This will help you debug the issue

        li_elements = WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.XPATH, "//ul[contains(@class, 'ds-flex ds-flex-col')]//li"))
            # EC.visibility_of_all_elements_located((By.XPATH, "//ul[contains(@class, 'ds-flex ds-flex-col')]//li"))
        )

        no_of_innings = len(li_elements)

        for i in range(no_of_innings):
            
            driver.get(url)

            time.sleep(2)

            # innings_dropdown = driver.find_element(By.XPATH, "//div[contains(@class, 'ds-popper-wrapper') and .//span[contains(@class, 'ds-text-tight-s')]]")
            # driver.execute_script("arguments[0].click();", innings_dropdown)

            innings_dropdown = driver.find_element(By.XPATH, "//div[@class='ds-popper-wrapper']//div[contains(@class, 'ds-flex ds-items-center ds-border-ui-stroke ds-h-6 ds-px-4 ds-border ds-bg-ui-fill ds-rounded-full ds-w-full ds-min-w-max ds-cursor-pointer')]")
            driver.execute_script("arguments[0].click();", innings_dropdown)
            # innings_dropdown.click()
            # time.sleep(2)
            # print(driver.page_source)
            li_elements = WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.XPATH, "//ul[contains(@class, 'ds-flex ds-flex-col')]//li"))
            )

            span_element = li_elements[i].find_element(By.XPATH, ".//span")
            driver.execute_script("arguments[0].click();", span_element)
            # continue
            buttons = driver.find_element(By.XPATH, "//span[contains(@class, 'ds-inline-flex') and .//span[text()='New']]")
            driver.execute_script("arguments[0].click();", buttons)
            driver.get_log('performance')
            # li_elements[2].click()
            # driver.execute_script("arguments[0].click();", li_elements[i])

            
            inn_name = str(match_id) + ".json"
            # print(inn_name)
            time.sleep(2)

            # driver.get_log('performance')
            scroll_page_func(driver, api_responses, api_url_types)

            folder_path = "espn_scrape_data/matches"
            os.makedirs(folder_path, exist_ok=True)
            file_path = os.path.join(folder_path, inn_name)

            with open(file_path, 'w') as f:
                json.dump(api_responses, f, indent=4)
                # json.dump({k: list(v) for k, v in api_url_types.items()}, f, indent=4)
                print(f"API URLs saved to {folder_path}/{inn_name}.", folder_path,inn_name)
            
            # api_responses = []
            
        # Setup scroll parameters
        

        # Save the categorized URLs to a file

        # Print out the differences
        print("\nDifferences in Comments API URLs:")
        print("Default Comments URLs:", len(api_url_types['comments_urls']))
        print("Other Comments URLs:", len(api_url_types['other_comments_urls']))

    finally:
        driver.quit()

def espn_scorecard_scrapper_engine(url, match_id):

    api_url_types = {
        'comments_urls': set(),
        'other_comments_urls': set()
    }

    api_responses = []
    

    driver = setup_driver()

    try:
        # Open the target webpage
        driver.get(url)

        time.sleep(2)

        inn_name = str(match_id) + "_scorecard.json"    
        # Wait until the element is present (modify the time as needed)
        wait = WebDriverWait(driver, 3)
        element = wait.until(

            EC.presence_of_all_elements_located(
                (By.XPATH, "//a[contains(@href, '/full-scorecard')]")
            )

        )

        # Click the element
        driver.execute_script("arguments[0].scrollIntoView(true);", element[1])
        driver.execute_script("arguments[0].click();", element[1])

        time.sleep(2)
        capture_and_categorize_urls(driver, api_responses, api_url_types, 1)

        folder_path = "espn_scrape_data/scorecard"
        os.makedirs(folder_path, exist_ok=True)
        file_path = os.path.join(folder_path, inn_name)

        with open(file_path, 'w') as f:
            json.dump(api_responses, f, indent=4)
            # json.dump({k: list(v) for k, v in api_url_types.items()}, f, indent=4)
            print(f"API URLs saved to {folder_path}/{inn_name}.", folder_path,inn_name)
        
        # api_responses = []
            
        print("\nDifferences in Comments API URLs:")
        print("Default Comments URLs:", len(api_url_types['comments_urls']))
        print("Other Comments URLs:", len(api_url_types['other_comments_urls']))

    finally:
        driver.quit()
        
# if __name__ == "__main__":
#     espn_commentary_scrapper_engine()

In [29]:
# espn_commentary_scrapper_engine(get_redirected_commentary_url(1432209, 2), 1432209)
# espn_scorecard_scrapper_engine(get_redirected_commentary_url(1428556, 1), 1428556)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# Wrapper function to integrate both steps
def process_match(match_id):
    # Step 1: Get the redirected commentary URL
    commentary_url = get_redirected_commentary_url(match_id, 2)
    # Step 2: Pass the URL and match ID to the scraper engine
    espn_commentary_scrapper_engine(commentary_url, match_id)


start_time = time.time()
# Run the scraping process in parallel
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
    executor.map(process_match, match_ids)

print(f"Total time taken: {time.time() - start_time} seconds")


Captured Fetch Request URL (ASC): https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?lang=en&seriesId=1442205&matchId=1442215&inningNumber=2&commentType=ALL&sortDirection=ASC&fromInningOver=38
Error capturing URL: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No resource with given identifier found"}
  (Session info: chrome=131.0.6778.139)
Stacktrace:
#0 0x566d60939efa <unknown>
#1 0x566d604626c0 <unknown>
#2 0x566d60449f63 <unknown>
#3 0x566d6044882c <unknown>
#4 0x566d604490cf <unknown>
#5 0x566d60449024 <unknown>
#6 0x566d60465c4f <unknown>
#7 0x566d604fee1a <unknown>
#8 0x566d604d4572 <unknown>
#9 0x566d604f2c4e <unknown>
#10 0x566d604d4313 <unknown>
#11 0x566d604a2fcf <unknown>
#12 0x566d604a3fee <unknown>
#13 0x566d609082df <unknown>
#14 0x566d6090c3fd <unknown>
#15 0x566d608f6977 <unknown>
#16 0x566d6090cb71 <unknown>
#17 0x566d608de96e <unknown>
#18 0x566d60928c18 <unknown>
#19 0x566d60928e1a <unknown>
#20 0x566d60938b6c <unknown>
#2

In [ ]:
# for i in match_ids:
#     espn_scorecard_scrapper_engine(get_redirected_commentary_url(i, 1), i)